In [ ]:
from dotenv import load_dotenv
from pyspark.sql.functions import col, when, count, round 
import os

load_dotenv()

True

In [16]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RetailEDA") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.1") \
    .config("fs.azure.account.key.dataintensiveproject.blob.core.windows.net", os.getenv("AZURE_STORAGE_KEY")) \
    .getOrCreate()

25/04/21 15:39:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [17]:
parquet_path = "wasbs://retail-data@dataintensiveproject.blob.core.windows.net/clean/retail_cleaned"

df = spark.read.parquet(parquet_path)
df.printSchema()
df.show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- InvoiceDate: timestamp_ntz (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- InvoiceYear: integer (nullable = true)
 |-- InvoiceMonth: integer (nullable = true)
 |-- InvoiceWeekday: integer (nullable = true)
 |-- InvoiceHour: integer (nullable = true)



+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+------------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|InvoiceYear|InvoiceMonth|InvoiceWeekday|InvoiceHour|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+------------+--------------+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|       2010|          12|             4|          8|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|       2010|          12|             4|          8|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|       2010|          12|             4|          8|
|   536365|   84029G|KNITTED UNION FLA..

In [20]:
# Basic row & column info
print(f"Total Rows: {df.count()}")
print(f"Total Columns: {len(df.columns)}")

# Check for nulls in each column
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

Total Rows: 397884
Total Columns: 12
+---------+---------+-----------+--------+-----------+---------+----------+-------+-----------+------------+--------------+-----------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|InvoiceYear|InvoiceMonth|InvoiceWeekday|InvoiceHour|
+---------+---------+-----------+--------+-----------+---------+----------+-------+-----------+------------+--------------+-----------+
|        0|        0|          0|       0|          0|        0|         0|      0|          0|           0|             0|          0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+-----------+------------+--------------+-----------+



In [30]:
#Top 10 Selling Products(by Quantity)

df.groupBy("Description") \
  .agg(count("Quantity").alias("TotalSold")) \
  .orderBy("TotalSold", ascending=False) \
  .show(10, truncate=False)

+----------------------------------+---------+
|Description                       |TotalSold|
+----------------------------------+---------+
|WHITE HANGING HEART T-LIGHT HOLDER|2028     |
|REGENCY CAKESTAND 3 TIER          |1723     |
|JUMBO BAG RED RETROSPOT           |1618     |
|ASSORTED COLOUR BIRD ORNAMENT     |1408     |
|PARTY BUNTING                     |1396     |
|LUNCH BAG RED RETROSPOT           |1316     |
|SET OF 3 CAKE TINS PANTRY DESIGN  |1159     |
|LUNCH BAG  BLACK SKULL.           |1105     |
|POSTAGE                           |1099     |
|PACK OF 72 RETROSPOT CAKE CASES   |1068     |
+----------------------------------+---------+
only showing top 10 rows



In [32]:
#Total sales by Country

df.groupBy("Country") \
  .agg(count("InvoiceNo").alias("TotalInvoices")) \
  .orderBy("TotalInvoices", ascending=False) \
  .show(truncate=False)

+---------------+-------------+
|Country        |TotalInvoices|
+---------------+-------------+
|United Kingdom |354321       |
|Germany        |9040         |
|France         |8341         |
|EIRE           |7236         |
|Spain          |2484         |
|Netherlands    |2359         |
|Belgium        |2031         |
|Switzerland    |1841         |
|Portugal       |1462         |
|Australia      |1182         |
|Norway         |1071         |
|Italy          |758          |
|Channel Islands|748          |
|Finland        |685          |
|Cyprus         |614          |
|Sweden         |451          |
|Austria        |398          |
|Denmark        |380          |
|Poland         |330          |
|Japan          |321          |
+---------------+-------------+
only showing top 20 rows



In [ ]:
#Sales Distribution by Year-Month

df.groupBy("InvoiceYear", "InvoiceMonth") \
  .count() \
  .orderBy("InvoiceYear", "InvoiceMonth") \
  .show(20)


+-----------+------------+-----+
|InvoiceYear|InvoiceMonth|count|
+-----------+------------+-----+
|       2010|          12|26157|
|       2011|           1|21229|
|       2011|           2|19927|
|       2011|           3|27175|
|       2011|           4|22642|
|       2011|           5|28320|
|       2011|           6|27185|
|       2011|           7|26825|
|       2011|           8|27007|
|       2011|           9|40028|
|       2011|          10|49554|
|       2011|          11|64531|
|       2011|          12|17304|
+-----------+------------+-----+



In [ ]:
#Sales by Weekday and Hour

df.groupBy("InvoiceWeekday", "InvoiceHour") \
  .count() \
  .orderBy("InvoiceWeekday", "InvoiceHour") \
  .show(20)


+--------------+-----------+-----+
|InvoiceWeekday|InvoiceHour|count|
+--------------+-----------+-----+
|             1|          9|   25|
|             1|         10| 3537|
|             1|         11|10217|
|             1|         12|13425|
|             1|         13|11673|
|             1|         14| 9878|
|             1|         15|10055|
|             1|         16| 3963|
|             2|          7|   61|
|             2|          8| 1564|
|             2|          9| 4348|
|             2|         10| 6132|
|             2|         11| 7504|
|             2|         12|10731|
|             2|         13|10595|
|             2|         14| 9212|
|             2|         15| 7705|
|             2|         16| 4063|
|             2|         17| 2978|
|             3|          7|   65|
+--------------+-----------+-----+
only showing top 20 rows



In [ ]:
#Total Revenue per Transaction (add a new column)

df = df.withColumn("TotalRevenue", round(col("Quantity") * col("UnitPrice"), 2))
df.select("InvoiceNo", "Quantity", "UnitPrice", "TotalRevenue").show(5)


+---------+--------+---------+------------+
|InvoiceNo|Quantity|UnitPrice|TotalRevenue|
+---------+--------+---------+------------+
|   536365|       6|     2.55|        15.3|
|   536365|       6|     3.39|       20.34|
|   536365|       8|     2.75|        22.0|
|   536365|       6|     3.39|       20.34|
|   536365|       6|     3.39|       20.34|
+---------+--------+---------+------------+
only showing top 5 rows



In [ ]:
#Revenue by Customer

df.groupBy("CustomerID") \
  .agg({"TotalRevenue": "sum"}) \
  .withColumnRenamed("sum(TotalRevenue)", "TotalSpent") \
  .orderBy("TotalSpent", ascending=False) \
  .show(10)


+----------+------------------+
|CustomerID|        TotalSpent|
+----------+------------------+
|   14646.0|280206.01999999984|
|   18102.0|259657.30000000005|
|   17450.0|194550.79000000007|
|   16446.0|          168472.5|
|   14911.0|143825.05999999944|
|   12415.0|124914.52999999975|
|   14156.0| 117379.6299999998|
|   17511.0| 91062.37999999996|
|   16029.0| 81024.84000000001|
|   12346.0|           77183.6|
+----------+------------------+
only showing top 10 rows



In [ ]:
#Most Purchased Products by Revenue

df.groupBy("Description") \
  .agg({"TotalRevenue": "sum"}) \
  .withColumnRenamed("sum(TotalRevenue)", "RevenueGenerated") \
  .orderBy("RevenueGenerated", ascending=False) \
  .show(10, truncate=False)


+----------------------------------+------------------+
|Description                       |RevenueGenerated  |
+----------------------------------+------------------+
|PAPER CRAFT , LITTLE BIRDIE       |168469.6          |
|REGENCY CAKESTAND 3 TIER          |142592.94999999963|
|WHITE HANGING HEART T-LIGHT HOLDER|100448.14999999866|
|JUMBO BAG RED RETROSPOT           |85220.78000000192 |
|MEDIUM CERAMIC TOP STORAGE JAR    |81416.72999999995 |
|POSTAGE                           |77803.95999999999 |
|PARTY BUNTING                     |68844.33000000037 |
|ASSORTED COLOUR BIRD ORNAMENT     |56580.33999999958 |
|Manual                            |53779.929999999964|
|RABBIT NIGHT LIGHT                |51346.200000000135|
+----------------------------------+------------------+
only showing top 10 rows



In [ ]:
#Monthly Revenue Trend

df.groupBy("InvoiceYear", "InvoiceMonth") \
  .agg({"TotalRevenue": "sum"}) \
  .orderBy("InvoiceYear", "InvoiceMonth") \
  .show(20)


+-----------+------------+------------------+
|InvoiceYear|InvoiceMonth| sum(TotalRevenue)|
+-----------+------------+------------------+
|       2010|          12| 572713.8900000163|
|       2011|           1| 569445.0400000077|
|       2011|           2| 447137.3500000165|
|       2011|           3|  595500.760000013|
|       2011|           4| 469200.3600000132|
|       2011|           5| 678594.5600000018|
|       2011|           6| 661213.6900000116|
|       2011|           7| 600091.0100000141|
|       2011|           8|  645343.900000009|
|       2011|           9| 952838.3799999963|
|       2011|          10|1039318.7899999822|
|       2011|          11|1161817.3799999433|
|       2011|          12| 518192.7900000037|
+-----------+------------+------------------+

